In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 13.2 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
     |████████████████████████████████| 880 kB 65.1 MB/s 
     |████████████████████████████████| 6.6 MB 75.9 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f38eb046beef9f2b5c127f2f69f6b4ae503f983e1352cafed4136075088ff7a9
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
%cd /content/gdrive/Shareddrives/DL

/content/gdrive/Shareddrives/DL


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [11]:
tc2code = pd.read_json('./data.jsonl',lines=True)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
print(tc2code.head())

                                            testCode  \
0  /*\n * This file was automatically generated b...   
1  /*\n * This file was automatically generated b...   
2  /*\n * This file was automatically generated b...   
3  /*\n * This file was automatically generated b...   
4  /*\n * This file was automatically generated b...   

                                          sourceCode  
0  package macaw.util;\n\nimport macaw.system.Mac...  
1  package macaw.util;\n\nimport macaw.system.Use...  
2  package macaw.util;\n\nimport java.util.regex....  
3  package macaw.util;\n\nimport java.awt.event.C...  
4  package macaw.util;\n\nimport javax.swing.*;\n...  


Random data Explorations

In [24]:
from transformers import T5Tokenizer, T5Model,T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small").to(device)
# model=T5Model.from_pretrained("Salesforce/codet5-small")
input_ids = tokenizer(
    tc2code['testCode'][0], return_tensors="pt"
).input_ids  # Batch size 1
decoder_input_ids = tokenizer(tc2code['sourceCode'][0], return_tensors="pt").input_ids  # Batch size 1

# forward pass
# outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
# last_hidden_states = outputs.last_hidden_state

Token indices sequence length is longer than the specified maximum sequence length for this model (1021 > 512). Running this sequence through the model will result in indexing errors


In [27]:
sample_outputs = model.generate(
    input_ids.to(device),
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

# outputs = model.generate(input_ids)
print("Generated docstring:", tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

Generated docstring:  public}if (label == "Labellabelfalse ; if}
{
}
public voidthis {({
(;;{



Define Tokenizer and Tokenize the input and op

In [28]:
# tc2code['token_lengths']=tc2code['sourceCode'].apply(lambda x: len(tokenizer.tokenize(x)))

In [29]:
# tc2code['token_lengths_test']=tc2code['testCode'].apply(lambda x: len(tokenizer.tokenize(x)))

In [30]:
# tc2code['token_lengths'].describe()

count     22947.000000
mean       1705.400488
std        3202.792489
min          13.000000
25%         419.500000
50%         832.000000
75%        1766.500000
max      112136.000000
Name: token_lengths, dtype: float64

The 75th percentile is 1766 hence we choose 1800 as the max_length

In [31]:
# tc2code['token_lengths_test'].describe()

count     22947.000000
mean       1007.715039
std        3212.269785
min          96.000000
25%         243.000000
50%         406.000000
75%         972.000000
max      274693.000000
Name: token_lengths_test, dtype: float64

The 75th percentile is 972 so lets go with 1766

In [45]:
def RobertaTokenizerForSeq2Seq(x):
  return tokenizer.batch_encode_plus(x,max_length=1800,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

In [61]:
def RobertaTokenizerForSeq2SeqTest(x):
  return tokenizer.batch_encode_plus(x,max_length=1000,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

In [43]:
print(RobertaTokenizerForSeq2Seq(tc2code['sourceCode'][0]))

tensor([[   1, 5610, 5318,  ...,    0,    0,    0]])


Report ඞ

In [44]:
sample_code=tc2code.head(10)
sample_code['test_Samples']=sample_code['testCode'].apply(lambda x: tokenizer.tokenize(x))
sample_code['source_Samples']=sample_code['sourceCode'].apply(lambda x: tokenizer.tokenize(x))
sample_code.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,testCode,sourceCode,token_lengths,token_lengths_test,test_Samples,source_Samples
0,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Mac...,1271,1019,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
1,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Use...,2658,2829,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
2,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport java.util.regex....,854,441,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
3,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport java.awt.event.C...,706,775,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
4,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport javax.swing.*;\n...,1453,312,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
5,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.businessLa...,4626,402,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
6,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Mac...,1150,872,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
7,/*\n * This file was automatically generated b...,/*\n * Created on 25-Jul-2009\n * Copyright (C...,3194,872,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[/*, Ċ, Ġ*, ĠCreated, Ġon, Ġ25, -, J, ul, -, 2..."
8,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Mac...,1150,2265,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[package, Ġmac, aw, ., util, ;, Ċ, Ċ, import, ..."
9,/*\n * This file was automatically generated b...,/*\n * Created on 25-Jul-2009\n * Copyright (C...,3194,2265,"[/*, Ċ, Ġ*, ĠThis, Ġfile, Ġwas, Ġautomatically...","[/*, Ċ, Ġ*, ĠCreated, Ġon, Ġ25, -, J, ul, -, 2..."


In [62]:
RobertaTokenizerForSeq2SeqTest(sample_code['testCode'].to_list())

{'input_ids': tensor([[    1, 20308,   203,  ...,    67,  1919,     2],
        [    1, 20308,   203,  ...,   225,  1071,     2],
        [    1, 20308,   203,  ...,     0,     0,     0],
        ...,
        [    1, 20308,   203,  ...,     0,     0,     0],
        [    1, 20308,   203,  ...,   225,  1071,     2],
        [    1, 20308,   203,  ...,   225,  1071,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

Training the models hopefully

In [63]:
X_train, X_test, y_train, y_test = train_test_split(tc2code['testCode'], tc2code['sourceCode'], test_size=0.2, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.1, shuffle=False)

In [64]:
X_train_tensors=RobertaTokenizerForSeq2SeqTest(X_train.to_list())
X_test_tensors=RobertaTokenizerForSeq2SeqTest(X_test.to_list())
X_val_tensors=RobertaTokenizerForSeq2SeqTest(X_val.to_list())

In [65]:
y_train_tensors=RobertaTokenizerForSeq2Seq(y_train.to_list())
y_test_tensors=RobertaTokenizerForSeq2Seq(y_test.to_list())
y_val_tensors=RobertaTokenizerForSeq2Seq(y_val.to_list())

In [95]:
input_ids_train = X_train_tensors['input_ids']
attention_masks_train = X_train_tensors['attention_mask']
labels_train = y_train_tensors['input_ids']

#validation set
input_ids_val = X_val_tensors['input_ids']
attention_masks_val = X_val_tensors['attention_mask']
labels_val = y_val_tensors['input_ids']


input_ids_test = X_test_tensors['input_ids']
attention_masks_test = X_test_tensors['attention_mask']
labels_test = y_test_tensors['input_ids']
expected_source_code=y_test.to_list()

In [96]:
expected_source_code[0]

'/*\n * Created on 14/mar/2010\n * Copyright (C) 2010 by Andrea Vacondio.\n *\n * This program is free software; you can redistribute it and/or modify it under the terms of the \n * GNU General Public License as published by the Free Software Foundation; \n * either version 2 of the License.\n * This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; \n * without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. \n * See the GNU General Public License for more details.\n * You should have received a copy of the GNU General Public License along with this program; \n * if not, write to the Free Software Foundation, Inc., \n *  59 Temple Place, Suite 330, Boston, MA 02111-1307 USA\n */\npackage org.pdfsam.guiclient.business.listeners;\n\nimport java.awt.event.InputEvent;\nimport java.awt.event.MouseWheelEvent;\nimport java.awt.event.MouseWheelListener;\n\nimport org.pdfsam.guiclient.gui.components.JPreviewImage;\n\n/**\n * 

In [67]:
import gc
gc.collect()

103

Training the model

In [70]:
t5_model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small",dropout_rate=0.5,d_model=1850,ignore_mismatched_sizes=True).to(device)
t5_model.to(device)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at Salesforce/codet5-small and are newly initialized because the shapes did not match:
- shared.weight: found shape torch.Size([32100, 512]) in the checkpoint and torch.Size([32100, 1850]) in the model instantiated
- encoder.embed_tokens.weight: found shape torch.Size([32100, 512]) in the checkpoint and torch.Size([32100, 1850]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.q.weight: found shape torch.Size([512, 512]) in the checkpoint and torch.Size([512, 1850]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.k.weight: found shape torch.Size([512, 512]) in the checkpoint and torch.Size([512, 1850]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.v.weight: found shape torch.Size([512, 512]) in the checkpoint and torch.Size([512, 1850]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.o.weight: found shape torch.Size([512, 512

T5ForConditionalGeneration(
  (shared): Embedding(32100, 1850)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 1850)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1850, out_features=512, bias=False)
              (k): Linear(in_features=1850, out_features=512, bias=False)
              (v): Linear(in_features=1850, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=1850, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.5, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1850, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=1850, bias=False)
              (dropo

In [97]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                             attention_masks_val, 
                             labels_val)

dataset_test = TensorDataset(input_ids_test, 
                             attention_masks_test, 
                             labels_test,
                             expected_source_code)

AttributeError: ignored

In [83]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 4

#train set
dataloader_train = DataLoader(dataset_train,
                              sampler = RandomSampler(dataset_train),
                              batch_size = batch_size)

#validation set
dataloader_val = DataLoader(dataset_val,
                              sampler = RandomSampler(dataset_val),
                              batch_size = batch_size)

dataloader_test = DataLoader(dataset_val,
                              sampler = RandomSampler(dataset_val),
                              batch_size = batch_size)

In [84]:
from transformers import get_linear_schedule_with_warmup

optimizer = torch.optim.AdamW(t5_model.parameters(),
                 lr = 1e-5,
                 eps = 1e-8)
                 
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0,
                                           num_training_steps = len(dataloader_train)*epochs)

In [91]:
def evaluate(dataloader_val,model):

    #evaluation mode 
    model.eval()
    
    #tracking variables
    loss_val_total = 0
    
    for batch in tqdm(dataloader_val):
        
        #load into GPU
        batch = tuple(b.to(device) for b in batch)
        
        #define inputs
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]}

        #compute logits
        with torch.no_grad():        
            outputs = model(**inputs)
        
        #compute loss
        loss = outputs[0]
        loss_val_total += loss.item()

    #compute average loss
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
            
    return loss_val_avg

In [92]:
def train_model(model):
    save_epoch=5
    validation_loss_epochs=[]
    training_loss_epochs=[]

    for epoch in tqdm(range(1, epochs+1)):
        
        model.train()
        
        loss_train_total = 0
        
        progress_bar = tqdm(dataloader_train, 
                            desc = 'Epoch {:1d}'.format(epoch), 
                            leave = False, 
                            disable = False)
        
        for batch in progress_bar:
            
            model.zero_grad() #set gradient to 0
        
            batch = tuple(b.to(device) for b in batch)
            
            inputs = {'input_ids': batch[0], 
                      'attention_mask': batch[1], 
                      'labels': batch[2]}
            
            outputs = model(**inputs) #unpack the dict straight into inputs
            
            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

        if epoch%save_epoch==0:    
          torch.save(model.state_dict(), f'Models/ CodeT5_ft_epoch{epoch}.model')
        
        tqdm.write('\n Epoch {epoch}')
        
        loss_train_ave = loss_train_total / len(dataloader_train)
        tqdm.write('Training loss: {loss_train_avg}')
        
        val_loss = evaluate(dataloader_val,model)
        validation_loss_epochs.append(val_loss)
        training_loss_epochs.append(loss_train_ave)
        
        tqdm.write(f'Validation loss: {val_loss}')
        return validation_loss_epochs,training_loss_epochs,model

In [87]:
val_losses,training_losses,t5_model=train_model(t5_model)

  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: ignored

In [93]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [88]:
def getPredictions(inputs,model):
      sample_outputs = model.generate(
        input_ids.to(device),
        do_sample=True, 
        max_length=50, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=3)
    # outputs = model.generate(input_ids)
      return tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

  def getPredictionForDataLoader()